## Collision élastique de deux disques

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox
import ipywidgets as widgets

from IPython import display
from IPython.display import Image, HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, ColumnDataSource, Select, TextInput
from bokeh.layouts import widgetbox, column, row
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

from time import sleep

output_notebook()

In [ ]:
Image("elastic_coll.png")

In [ ]:
# fonction pour calculer la masse
mass = lambda P : (4 / 3) * P['density'] * np.pi * P['radius'] ** 3

vel_ = lambda P: (P['v']*np.cos(P['alpha']*np.pi/180), P['v']*np.sin(P['alpha']*np.pi/180))

traj_ = lambda P:  [P['x'] + P['v_x']*np.linspace(0, 500, 500)/10, P['y'] + P['v_y']*np.linspace(0, 500, 500)/10]

new_ = lambda P, dt: (P['x'] + P['v_x']*dt, P['y'] + P['v_y']*dt)

dist_ = lambda P1, P2: math.sqrt((P1['x'] - P2['x'])**2 + (P1['y'] - P2['y'])**2)

proj_v = lambda P, w: (P['v_x']*w[0] + P['v_y']*w[1])/(np.linalg.norm(w))**2

proj_v_vec = lambda P, w: proj_v(P,w)*w

e_cinetique = lambda P1, P2: P1['m']*P1['v']**2 + P2['m']*P2['v']**2

quant_mov_u = lambda P1, P2, u: P1['m']*proj_v(P1, u) + P2['m']*proj_v(P2,u)

new_v_coll = lambda P, w, u: (proj_v_vec(P,w) - proj_v_vec(P,u))

In [ ]:

def update_values_time(P1, P2, dt, scenario):
    t = 0
    P1['traj'] = [(P1['x'], P1['y'])]
    P2['traj'] = [(P2['x'], P2['y'])]
    
    P1['init_x'] = P1['x']
    P1['init_y'] = P1['y']
    P2['init_x'] = P2['x']
    P2['init_y'] = P2['y']
    
    while t < min(P1['t'], P2['t']):
        
        # calculer prochain possition
        update_position(P1, P2, dt)
        
        P1['traj'].append((P1['x'], P1['y']))
        P2['traj'].append((P2['x'], P2['y']))

        # verifier si il y a une collision entre P1 et P2
        if dist_(P1, P2) < (P1['radius'] + P2['radius']):            
            update_velocity(P1, P2)
        
        # verifier si il y a collision avec les limites du scenario
        verifier_limites(P1, scenario)
        verifier_limites(P2, scenario)

        t += dt  
    
    P1['traj'] = np.array(P1['traj'])
    P2['traj'] = np.array(P2['traj'])




def update_position(P1, P2 ,dt):
    """ calculer la nouvelle position"""
    P1['x'], P1['y'] = new_(P1, dt)
    P2['x'], P2['y'] = new_(P2, dt)


def update_velocity(P1, P2):
    """ calculer la velocité après collision entre les deux particules"""
    # calculer vecteur perpendiculaire à la collision
    u = np.array([P2['x'] - P1['x'], P2['y'] - P1['y']])/dist_(P1,P2)

    # point collision
    p_x = P1['x'] + u[0]*P1['radius']/dist_(P1,P2)
    p_y = P1['y'] + u[1]*P1['radius']/dist_(P1,P2)

    # vecteur tangent à la collision
    w = np.array([u[1], -u[0]])

    # mettre à jour velocité après collision
    P1['v_x'], P1['v_y'] = new_v_coll(P1, w, u)
    P2['v_x'], P2['v_y'] = new_v_coll(P2, w, u)

    P1['v'] = math.sqrt(P1['v_x']**2 + P1['v_y']**2)
    P2['v'] = math.sqrt(P2['v_x']**2 + P2['v_y']**2)

    

def verifier_limites(P, scenario):
    """ calculer velocité après collision entre particules et un des limites du scénario"""
    
    col = False # boolean pour indiquer si il y a collision ou pas 
    
    # verifier si il y a une collision entre P et limite superiéure
    if P['y'] >= scenario['y_MAX'] - P['radius'] - 5: # -5 pour épaisseur de la ligne
        col = True
        u = np.array([0, -1])
        w = np.array([1, 0])

        
    # verifier si il y a une collision entre P et limite inferiéure
    elif P['y'] <= scenario['y_MIN'] + P['radius'] +5:
        col = True
        u = np.array([0, 1])
        w = np.array([-1, 0])


    # verifier si il y a une collision entre P et limite droite
    elif P['x'] >= scenario['x_MAX'] - P['radius'] -5:
        col = True
        u = np.array([-1, 0])
        w = np.array([0, 1])


    # verifier si il y a une collision entre P et limite gauche
    elif P['x'] <= scenario['x_MIN'] + P['radius'] +5:
        col = True
        u = np.array([1, 0])
        w = np.array([0, -1])

    # si il y a eu une collision, mettre à jour la velocité 
    if col:
        P['v_x'], P['v_y'] = new_v_coll(P, w, u)
        P['v'] = math.sqrt(P['v_x']**2 + P['v_y']**2)
    


def centre_grav(P1, P2, idx=0):
    """ calculer le centre de gravité"""
    m_tot = (P1['radius'] + P2['radius'])
    c_x = P1['traj'][idx,0]*P1['radius']/m_tot + P2['traj'][idx,0]*P2['radius']/m_tot
    c_y = P1['traj'][idx,1]*P1['radius']/m_tot + P2['traj'][idx,1]*P2['radius']/m_tot
    return c_x, c_y
    
    

def update_params(P, P_y, P_radius, v_P, alpha_P):
    P['y'] = P_y
    P['v'] = v_P
    
    if P['name'] == 'A':
        P['alpha'] = alpha_P -90
    else:
        P['alpha'] = alpha_P + 90
    
    P['radius'] = P_radius
    P['m'] = mass(P)
    P['v_x'], P['v_y'] = vel_(P)
    #P['traj'] = traj_(P)


def convert_to_array(P):
    for k in ['x', 'init_x', 'y', 'init_y']:
        P[k] = np.array(P[k])

    

# Initialize
def initialize(t, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b):
    A = {'x':x_a, 'y': y_a, 'v':v_a, 'alpha': alpha_a, 'density': 1, 'radius': rad_a, 't':t, 'name':'A'}
    B = {'x':x_b, 'y': y_b, 'v':v_b, 'alpha': alpha_b, 'density': 1, 'radius': rad_b, 't':t, 'name':'B'}


    A['v_x'], A['v_y'] = vel_(A) 
    B['v_x'], B['v_y'] = vel_(B) 

    A['m'], B['m'] = mass(A), mass(B)
    
    return A,B


In [ ]:
def create_figure():
    global T, x_a, x_b

    T = 15
    x_a = -100
    x_b = 100
    
    y_a = 0
    y_b = 0

    v_a = 10
    v_b = 5
    
    Y_MAX = 100
    Y_MIN = -100
    X_MAX = 200
    X_MIN = -200
    
    scenario = {'y_MAX': Y_MAX, 'y_MIN': Y_MIN, 'x_MAX': X_MAX, 'x_MIN': X_MIN}

    A, B = initialize(T, x_a, x_b, y_a, y_b, v_a, v_b, 180, 0, 3, 6) #t, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b
    update_values_time(A, B, 0.1, scenario)
    CG_x, CG_y = centre_grav(A,B)
    
    # Figure
    p = figure(title="Collision", plot_height=450, plot_width=900, y_range=(scenario['y_MIN'], scenario['y_MAX']), 
               x_range=(scenario['x_MIN'], scenario['x_MAX']), 
               background_fill_color='#ffffff')

    # grille
    p.ygrid.minor_grid_line_color = 'grey'
    p.ygrid.minor_grid_line_alpha = 0.3

    p.xgrid.minor_grid_line_color = 'grey'
    p.xgrid.minor_grid_line_alpha = 0.3

    # limites du scénario
    p.line(x=np.linspace(scenario['x_MIN'],scenario['x_MAX'], 3), y=[scenario['y_MIN']]*3, line_color='#000000', line_width=10)
    p.line(x=np.linspace(scenario['x_MIN'],scenario['x_MAX'], 3), y=[scenario['y_MAX']]*3, line_color='#000000', line_width=10)
    p.line(x=[scenario['x_MIN']]*3, y=np.linspace(scenario['y_MIN'],scenario['y_MAX'], 3), line_color='#000000', line_width=10)
    p.line(x=[scenario['x_MAX']]*3, y=np.linspace(scenario['y_MIN'],scenario['y_MAX'], 3), line_color='#000000', line_width=10)
    

    source_A = ColumnDataSource(data= {'x_t': A['traj'][:,0], 'y_t': A['traj'][:,1]})
    source_B = ColumnDataSource(data= {'x_t': B['traj'][:,0], 'y_t': B['traj'][:,1]})

    P_A = p.circle(x=A['init_x'], y=A['init_y'], size=A['radius'], fill_color='#e32020', line_color='#e32020', legend=A['name'])
    P_B = p.circle(x=B['init_x'], y=B['init_y'], size=B['radius'], fill_color='#0ABDE3', line_color='#0ABDE3', legend=B['name'])

    g_pA = P_A.glyph
    g_pB = P_B.glyph

    # centre de gravité du système de masses 
    CG = p.circle(x=[CG_x], y=[CG_y], size=3, fill_color='#000000', line_color='#000000')
    g_CG = CG.glyph
    

    Traj_A = p.line(x=A['traj'][:,0], y=A['traj'][:,1], color='#e32020')
    Traj_B = p.line(x=B['traj'][:,0], y=B['traj'][:,0], color='#0ABDE3')


    def update(A_x, B_x, A_y, B_y, A_radius, B_radius, v_A, v_B, alpha_A, alpha_B, play_b):
        dt = 0.1
        
        A, B = initialize(T, A_x, B_x, A_y, B_y, v_A, v_B, alpha_A, alpha_B, A_radius, B_radius)
        update_values_time(A, B, dt, scenario)
        
        CG_x, CG_y = centre_grav(A,B)

        source_A = ColumnDataSource(data= {'x_t': A['traj'][:,0], 'y_t': A['traj'][:,1]})
        source_B = ColumnDataSource(data= {'x_t': B['traj'][:,0], 'y_t': B['traj'][:,1]})

        # mettre à jour radius
        g_pA.size = A['radius']*4 
        g_pB.size = B['radius']*4


        # mettre à jour position
        g_pA.y = A_y
        g_pB.y = B_y
        
        g_pA.x = A_x
        g_pB.x = B_x

        g_CG.x = CG_x
        g_CG.y = CG_y
        
        # mettre à jour trajectoire
        Traj_A.data_source.data['x'] = source_A.data['x_t']
        Traj_A.data_source.data['y'] = source_A.data['y_t']
        Traj_B.data_source.data['x'] = source_B.data['x_t']
        Traj_B.data_source.data['y'] = source_B.data['y_t']

        print('Masse point A: {:0.2f} kg'.format(A['m']/1000))
        print('Masse point B: {:0.2f} kg'.format(B['m']/1000))
        
        if play_b:
            
            t = 0
            idx = 0
            while t < A['t']:
                
            
                sleep(dt)
                g_pA.x = source_A.data['x_t'][idx]
                g_pA.y = source_A.data['y_t'][idx]
                
                g_pB.x = source_B.data['x_t'][idx]
                g_pB.y = source_B.data['y_t'][idx]
                
                CG_x, CG_y = centre_grav(A, B, idx)
                g_CG.x = CG_x
                g_CG.y = CG_y
                
                
                t += dt
                idx += 1
                
                push_notebook() 
                       

        push_notebook()

    slider_Ax = widgets.IntSlider(min=-180, max=180, step=5, value=-100, description='$A_{x} [m]$:')
    slider_Bx = widgets.IntSlider(min=-180, max=180, step=5, value=40, description='$B_{x} [m]$:')
    
    slider_Ay = widgets.IntSlider(min=-80, max=80, step=5, value=0, description='$A_{y} [m]$:')
    slider_By = widgets.IntSlider(min=-80, max=80, step=5, value=-40, description='$B_{y} [m]$:')

    slider_Ar = widgets.FloatSlider(min=2,max=15,step=1,value=3, description='$A_{radius} [m]$:')
    slider_Br = widgets.FloatSlider(min=2,max=15,step=1,value=10, description='$B_{radius} [m]$:')

    slider_Av = widgets.FloatSlider(min=2,max=40,step=1,value=11, description='$v_{A} [m/s]$:')
    slider_Bv = widgets.FloatSlider(min=2,max=40,step=1,value=35, description='$v_{B} [m/s]$:')

    slider_A_alpha = widgets.FloatSlider(min=0,max=360,step=1,value=15, description='$ alpha A [º]$:')
    slider_B_alpha = widgets.FloatSlider(min=0,max=360,step=1,value=110, description='$ alpha B [º]$:')

    play_b = widgets.ToggleButton(value=False, description='Play Animation',disabled=False)

    VB_A = VBox([slider_Ax, slider_Ay, slider_Ar, slider_Av, slider_A_alpha])
    VB_B = VBox([slider_Bx, slider_By, slider_Br, slider_Bv, slider_B_alpha])
    HB = HBox([VB_A, VB_B])
    
    show(p, notebook_handle=True)
    
    interact(update, A_x = HB.children[0].children[0], B_x = HB.children[1].children[0],
             A_y = HB.children[0].children[1], B_y = HB.children[1].children[1],
             A_radius = HB.children[0].children[2], B_radius = HB.children[1].children[2],
            v_A =HB.children[0].children[3], v_B = HB.children[1].children[3],
             alpha_A = HB.children[0].children[4], alpha_B = HB.children[1].children[4], play_b=play_b);




In [ ]:
create_figure()

In [ ]:
# TODO: 

# SOLVE PROBLEM WITH SIZE OF BALLS (SOLVED aprox.)

# HOW TO HANDLE END OF ANIMATION -- DO NOT RECOVER INITIAL POSITION - 
# --> RIGHT NOW IT IS NECESSARY TO RECLICK THE BUTTON TO RECOVER INITIAL POSITION

# HOW TO SAVE PLOT

# DISPOSITION OF WIDGETS! 

# ADD
    # CENTER OF MASS (Done)
    # SPEED OF CENTER OF MASS
    # COLLISION WITH BORDERS (Done)
    
    
